In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
import joblib

In [2]:
# Load the dataset
data = pd.read_csv(r"C:\Users\Dell\Downloads\anxietynurture.csv")

In [3]:
data.head()

,Do you feel nervous?,Do you worry a lot?,Do you feel anxious about the future?,Do you have touble relaxing?,Do you find it hard to sit in one place for a while?,Do you get irritated easily?,Do you feel something scary might happen to you?,Anxiety Risk
0,1,2,0,0,0,2,1,Mild Anxiety
1,3,3,3,0,2,1,2,Moderate Anxiety
2,0,2,3,1,2,2,2,Moderate Anxiety
3,1,3,3,2,0,2,3,Moderate Anxiety
4,1,1,3,3,3,0,0,Moderate Anxiety


In [4]:
# Extracting categorical columns
categorical_cols = ['Do you feel nervous?',
                    'Do you worry a lot?',
                    'Do you feel anxious about the future?',
                    'Do you have touble relaxing?',
                    'Do you find it hard to sit in one place for a while?',
                    'Do you get irritated easily?',
                    'Do you feel something scary might happen to you?']

In [5]:
# One-hot encode categorical variables
encoder = OneHotEncoder(sparse=False, drop='first')
encoded_data = encoder.fit_transform(data[categorical_cols])

C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
# Create a DataFrame with encoded features
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))

In [7]:
# Combine encoded features with numerical features
data_encoded = pd.concat([encoded_df, data.drop(columns=categorical_cols)], axis=1)

In [8]:
import numpy as np

# Check for NaN values in the entire DataFrame
if data_encoded.isnull().values.any():
    # Fill NaN values with random integers between 0 and 3
    fill_values = {col: np.random.randint(0, 4) for col in data_encoded.columns}
    data_encoded.fillna(fill_values, inplace=True)

In [9]:
# Separate features (X) and target variable (y)
X = data_encoded.drop(columns=['Anxiety Risk'])
y = data_encoded['Anxiety Risk']

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Initialize SVM classifier
svm_classifier = SVC(kernel='rbf', random_state=42)

In [12]:
# Train the SVM model
svm_classifier.fit(X_train, y_train)

SVC(random_state=42)

In [13]:
# Predict on the test set using SVM
svm_y_pred = svm_classifier.predict(X_test)

In [14]:
# Evaluate the SVM model
print("\nSVM Confusion Matrix:")
print(confusion_matrix(y_test, svm_y_pred))
print("\nSVM Classification Report:")
print(classification_report(y_test, svm_y_pred, zero_division=1))


SVM Confusion Matrix:
[[359   0  24   0]
 [ 16   8   0   0]
 [  5   0 534   1]
 [  0   0  39  55]]

SVM Classification Report:
                  precision    recall  f1-score   support

    Mild Anxiety       0.94      0.94      0.94       383
 Minimal Anxiety       1.00      0.33      0.50        24
Moderate Anxiety       0.89      0.99      0.94       540
  Severe Anxiety       0.98      0.59      0.73        94

        accuracy                           0.92      1041
       macro avg       0.96      0.71      0.78      1041
    weighted avg       0.92      0.92      0.91      1041



In [15]:
# Save the trained SVM classifier model
joblib.dump(svm_classifier, 'svman_model.pkl')

['svman_model.pkl']

In [16]:
# Save the OneHotEncoder
joblib.dump(encoder, 'oneam_hot_encoder.pkl')

['oneam_hot_encoder.pkl']

Testing

In [17]:
import joblib
import pandas as pd

In [18]:
# Load the saved SVM model
loaded_svm_model = joblib.load('svman_model.pkl')
# Load the saved OneHotEncoder
loaded_encoder = joblib.load('oneam_hot_encoder.pkl')

In [31]:
# Prepare the input data
input_data = pd.DataFrame({
    'Do you feel nervous?': [0],
    'Do you worry a lot?': [0],
    'Do you feel anxious about the future?': [0],
    'Do you have touble relaxing?': [2],
    'Do you find it hard to sit in one place for a while?': [1],
    'Do you get irritated easily?': [1],
    'Do you feel something scary might happen to you?': [2]
})

In [32]:
# One-hot encode the input data
encoded_input = loaded_encoder.transform(input_data)

In [33]:
# Make predictions using the loaded SVM model
predictions = loaded_svm_model.predict(encoded_input)

C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [34]:
print("Predictions:", predictions)

Predictions: ['Mild Anxiety']
